Paso 1: Primera función

In [1]:
import pydicom as dcm
from pathlib import Path
import tkinter

def Load_Slices(directorio):
    informacionDICOM=[]
    for nombre in Path(directorio).glob("*.dcm"):
      print(f"Loading: {nombre}")
      informacionDICOM.append(dcm.dcmread(nombre))

    return informacionDICOM

In [2]:
from tkinter import filedialog
foldername=filedialog.askdirectory(title='')#se abre una ventana emergente que permite al usuario seleccionar un directorio
print(foldername)
Load_Slices(foldername)

C:/Users/usuario/Downloads/DICOMs/DICOMs/DICOM_001
Loading: C:\Users\usuario\Downloads\DICOMs\DICOMs\DICOM_001\1-001.dcm
Loading: C:\Users\usuario\Downloads\DICOMs\DICOMs\DICOM_001\1-002.dcm
Loading: C:\Users\usuario\Downloads\DICOMs\DICOMs\DICOM_001\1-003.dcm
Loading: C:\Users\usuario\Downloads\DICOMs\DICOMs\DICOM_001\1-004.dcm
Loading: C:\Users\usuario\Downloads\DICOMs\DICOMs\DICOM_001\1-005.dcm
Loading: C:\Users\usuario\Downloads\DICOMs\DICOMs\DICOM_001\1-006.dcm
Loading: C:\Users\usuario\Downloads\DICOMs\DICOMs\DICOM_001\1-007.dcm
Loading: C:\Users\usuario\Downloads\DICOMs\DICOMs\DICOM_001\1-008.dcm
Loading: C:\Users\usuario\Downloads\DICOMs\DICOMs\DICOM_001\1-009.dcm
Loading: C:\Users\usuario\Downloads\DICOMs\DICOMs\DICOM_001\1-010.dcm
Loading: C:\Users\usuario\Downloads\DICOMs\DICOMs\DICOM_001\1-011.dcm
Loading: C:\Users\usuario\Downloads\DICOMs\DICOMs\DICOM_001\1-012.dcm
Loading: C:\Users\usuario\Downloads\DICOMs\DICOMs\DICOM_001\1-013.dcm
Loading: C:\Users\usuario\Downloads\DIC

[Dataset.file_meta -------------------------------
 (0002, 0000) File Meta Information Group Length  UL: 206
 (0002, 0001) File Meta Information Version       OB: b'\x00\x01'
 (0002, 0002) Media Storage SOP Class UID         UI: CT Image Storage
 (0002, 0003) Media Storage SOP Instance UID      UI: 1.3.6.1.4.1.14519.5.2.1.99.1071.18421685298298271418159236661666
 (0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
 (0002, 0012) Implementation Class UID            UI: 1.3.6.1.4.1.22213.1.143
 (0002, 0013) Implementation Version Name         SH: '0.5'
 (0002, 0016) Source Application Entity Title     AE: 'POSDA'
 -------------------------------------------------
 (0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
 (0008, 0008) Image Type                          CS: ['DERIVED', 'PRIMARY', 'MPR']
 (0008, 0016) SOP Class UID                       UI: CT Image Storage
 (0008, 0018) SOP Instance UID                    UI: 1.3.6.1.4.1.14519.5.2.1.99.1

In [5]:
def OrientacionCorte (dato):#ds son los datos sacados del DICOM
    orientacion=dato.ImageOrientationPatient #es un atributo que describe la orientación espacial de la imagen,utilizando 6 valores que representan 2 vectores ortogonales
    redondeo = [round(x) for x in orientacion] #redondea los valores del vector iop para eliminar imprecisiones numericas y trabajar con valores más claros (cercanos a 0 o 1)
    plano_cruzado = np.cross(redondeo[0:3], redondeo[3:6]) #calcula el producto cruzado entre los 2 vectores. Genera un nuevo vector perpendicular a los 2 originales, que corresponde al eje en el que está cortada la imagen
    plano = [abs(x) for x in plano_cruzado] #toma el valor absoluto de cada componente del vector para poder comparar
    if plano[0] == 1: #verifica que componente del vector es dominante para determinar la orientación del plano (mayor valor absoluto)
        return 'Sagital'
    elif plano[1] == 1:
        return 'Coronal'
    elif plano[2] == 1:
        return 'Axial'
    else:
        return 'NA'

In [6]:
def datosHounsfield(slices):
    datos_Hounsfield=[]
    for dato in slices:
        matriz_pixel=dato.pixel_array
        m = dato.RescaleSlope if 'RescaleSlope' in dato else 1
        b = dato.RescaleIntercept if 'RescaleIntercept' in dato else 0
        operacion= (matriz_pixel * m) + b
        datos_Hounsfield.append(operacion)
    return datos_Hounsfield


In [7]:
import numpy as np

datos = Load_Slices(foldername)  

def creaVolumen(datos):
    slices = []
    for i in datos:
        if hasattr(i, "ImagePositionPatient"):
            slices.append(i)
    
    # Obtener el tamaño del píxel y el grosor de la sección
    ps = slices[0].PixelSpacing  
    ss = slices[0].SliceThickness  
    
    # Obtener el tipo de corte
    tipo_corte = OrientacionCorte(slices[0])
    
    # Convertir los datos de los cortes a unidades Hounsfield
    datos_HU = datosHounsfield(slices)

    if tipo_corte == 'Axial':
        print("Adquisición axial")
        slices = sorted(slices, key=lambda s: s.ImagePositionPatient[2])
        print(f"La secuencia se compone de {len(slices)} cortes")
        img_shape = list(slices[0].pixel_array.shape)
        img_shape.append(len(slices))
        img3d = np.zeros(img_shape)
        for i, s in enumerate(slices):
            img3d[:, :, len(slices)-i-1] = datos_HU[i]  # Usar datos_HU en lugar de s.pixel_array
        ax_aspect = ps[0] / ps[1]
        sag_aspect = ss / ps[1]
        cor_aspect = ss / ps[0]
        voxel_size = [ps[0], ps[1], ss]
    elif tipo_corte == 'Coronal':
        print("Adquisición Coronal")
        slices = sorted(slices, key=lambda s: s.ImagePositionPatient[1])
        print(f"La secuencia se compone de {len(slices)} cortes")
        img_shape = []
        img_shape.append(len(slices))
        img_shape.append(slices[0].pixel_array.shape[0])
        img_shape.append(slices[0].pixel_array.shape[1])
        img3d = np.zeros(img_shape)
        for i, s in enumerate(slices):
            img3d[i, :, :] = datos_HU[i].T  # Usar datos_HU
        ax_aspect = ss / ps[1]
        sag_aspect = ps[0] / ss
        cor_aspect = ps[0] / ps[1]
        voxel_size = [ps[0], ss, ps[1]]
    
    elif tipo_corte == 'Sagital':
        print("Adquisición Sagital")
        slices = sorted(slices, key=lambda s: s.ImagePositionPatient[0])
        print(f"La secuencia se compone de {len(slices)} cortes")
        img_shape = []
        img_shape.append(slices[0].pixel_array.shape[0])
        img_shape.append(len(slices))    
        img_shape.append(slices[0].pixel_array.shape[1])
        img3d = np.zeros(img_shape)
        for i, s in enumerate(slices):
            img3d[:, len(slices)-i-1, :] = datos_HU[i].T  # Usar datos_HU
        ax_aspect = ps[0] / ss
        sag_aspect = ps[1] / ps[0] 
        cor_aspect = ps[1] / ss
        voxel_size = [ss, ps[0], ps[1]]
    else:
        print("Error en la obtención de la vista")  
        
    return np.array(datos), img3d.shape, (ax_aspect, sag_aspect, cor_aspect), voxel_size


Loading: C:\Users\usuario\Downloads\DICOMs\DICOMs\DICOM_001\1-001.dcm
Loading: C:\Users\usuario\Downloads\DICOMs\DICOMs\DICOM_001\1-002.dcm
Loading: C:\Users\usuario\Downloads\DICOMs\DICOMs\DICOM_001\1-003.dcm
Loading: C:\Users\usuario\Downloads\DICOMs\DICOMs\DICOM_001\1-004.dcm
Loading: C:\Users\usuario\Downloads\DICOMs\DICOMs\DICOM_001\1-005.dcm
Loading: C:\Users\usuario\Downloads\DICOMs\DICOMs\DICOM_001\1-006.dcm
Loading: C:\Users\usuario\Downloads\DICOMs\DICOMs\DICOM_001\1-007.dcm
Loading: C:\Users\usuario\Downloads\DICOMs\DICOMs\DICOM_001\1-008.dcm
Loading: C:\Users\usuario\Downloads\DICOMs\DICOMs\DICOM_001\1-009.dcm
Loading: C:\Users\usuario\Downloads\DICOMs\DICOMs\DICOM_001\1-010.dcm
Loading: C:\Users\usuario\Downloads\DICOMs\DICOMs\DICOM_001\1-011.dcm
Loading: C:\Users\usuario\Downloads\DICOMs\DICOMs\DICOM_001\1-012.dcm
Loading: C:\Users\usuario\Downloads\DICOMs\DICOMs\DICOM_001\1-013.dcm
Loading: C:\Users\usuario\Downloads\DICOMs\DICOMs\DICOM_001\1-014.dcm
Loading: C:\Users\us

In [8]:
creaVolumen(datos)

Adquisición Sagital
La secuencia se compone de 126 cortes


(array([Dataset.file_meta -------------------------------
        (0002, 0000) File Meta Information Group Length  UL: 206
        (0002, 0001) File Meta Information Version       OB: b'\x00\x01'
        (0002, 0002) Media Storage SOP Class UID         UI: CT Image Storage
        (0002, 0003) Media Storage SOP Instance UID      UI: 1.3.6.1.4.1.14519.5.2.1.99.1071.18421685298298271418159236661666
        (0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
        (0002, 0012) Implementation Class UID            UI: 1.3.6.1.4.1.22213.1.143
        (0002, 0013) Implementation Version Name         SH: '0.5'
        (0002, 0016) Source Application Entity Title     AE: 'POSDA'
        -------------------------------------------------
        (0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
        (0008, 0008) Image Type                          CS: ['DERIVED', 'PRIMARY', 'MPR']
        (0008, 0016) SOP Class UID                       UI: CT Image